Necesitamos crear un código que extraiga la información de las tablas del pdf. Intenté utilizar pdfplumber pero me percaté que las tablas están pegadas como imágenes, por lo que considero necesario utilizar un nuevo método. Te comparto el pdf y además un código que había utilizado para descargar información de un pdf que estaba mucho más limpio y tenía sus tablas como tablas a diferencia de este. El archivo también es para que te guíes de la estructura del archivo que quiero recibir. Por el momento la parte de los paneles no es necesaria, simplemente quiero obtener la información en formato excel para tenerla visualizable y usable como base de datos.

In [7]:
pip install opencv-python

   ---------------------------------------- 0.0/39.0 MB ? eta -:--:--
   -- ------------------------------------- 2.1/39.0 MB 14.6 MB/s eta 0:00:03
   ---- ----------------------------------- 4.2/39.0 MB 11.8 MB/s eta 0:00:03
   -------- ------------------------------- 8.1/39.0 MB 14.4 MB/s eta 0:00:03
   ------------ --------------------------- 11.8/39.0 MB 15.2 MB/s eta 0:00:02
   ------------- -------------------------- 13.4/39.0 MB 15.6 MB/s eta 0:00:02
   ------------- -------------------------- 13.6/39.0 MB 11.7 MB/s eta 0:00:03
   -------------- ------------------------- 14.4/39.0 MB 10.7 MB/s eta 0:00:03
   ---------------- ----------------------- 16.3/39.0 MB 10.0 MB/s eta 0:00:03
   -------------------- ------------------- 19.7/39.0 MB 10.8 MB/s eta 0:00:02
   ----------------------- ---------------- 23.3/39.0 MB 11.6 MB/s eta 0:00:02
   ------------------------ --------------- 23.9/39.0 MB 10.6 MB/s eta 0:00:02
   -------------------------- ------------- 25.7/39.0 MB 10.4 MB

  You can safely remove it manually.
  You can safely remove it manually.


In [ ]:
import pytesseract
from pdf2image import convert_from_path
import pandas as pd
import re
import os
import sys

# ==========================================
# 1. CONFIGURACIÓN DE RUTAS (AJUSTA AQUÍ)
# ==========================================

# Ruta al ejecutable de Tesseract (donde lo instalaste)
# Si no está aquí, busca "tesseract.exe" en tu PC y pega la ruta.
TESSERACT_CMD = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

# Ruta a la carpeta 'bin' de Poppler (la que descomprimiste y copiaste)
# IMPORTANTE: Asegúrate de que esta ruta termine en \bin
POPPLER_PATH = r'C:/Program Files/Poppler/poppler-25.12.0/Library/bin'

# Archivo de entrada y salida
PDF_PATH = "../data/raw/20251209-V.pdf"
OUTPUT_EXCEL = "../data/intermediate/LIGIE_Tabla_Final.xlsx"

# ==========================================
# 2. CONFIGURACIÓN DEL MOTOR OCR
# ==========================================

# Validamos que las herramientas existan antes de empezar
if not os.path.exists(TESSERACT_CMD):
    print(f"ERROR: No encuentro Tesseract en: {TESSERACT_CMD}")
    print("Por favor instala Tesseract o corrige la ruta en el código.")
    sys.exit()

if not os.path.exists(POPPLER_PATH):
    print(f"ERROR: No encuentro la carpeta bin de Poppler en: {POPPLER_PATH}")
    print("Por favor verifica donde descomprimiste Poppler.")
    sys.exit()

# Conectamos Tesseract a Python
pytesseract.pytesseract.tesseract_cmd = TESSERACT_CMD

# Expresiones Regulares (El "cerebro" del script)
# Detecta el inicio de una fracción: 8 dígitos (ej: 8409.99.14)
REGEX_CODIGO = re.compile(r"^(\d{4}\.\d{2}\.\d{2})")

# Lista de textos basura para limpiar encabezados y pies de página
TEXTOS_IGNORAR = [
    "CÁMARA DE", "DIPUTADOS", "LXVI LEGISLATURA", "Comisión de Economía", 
    "DICTAMEN RELATIVO", "Página", "CÓDIGO", "DESCRIPCIÓN", "UNIDAD", 
    "INICIATIVA", "EJECUTIVO", "FEDERAL", "CUOTA", "(ARANCEL)", 
    "MODIFICACIÓN", "DICTAMEN", "Hoja", "Anexo", "The following table"
]

# ==========================================
# 3. FUNCIONES DE PROCESAMIENTO
# ==========================================

def es_linea_basura(texto):
    """Filtra encabezados, pies de página y líneas vacías."""
    if not texto: return True
    if len(texto) < 4: return True # Líneas muy cortas suelen ser basura o números de página
    
    texto_upper = texto.upper()
    for basura in TEXTOS_IGNORAR:
        if basura.upper() in texto_upper:
            return True
    return False

def procesar_registro(codigo, descripcion_sucia):
    """
    Toma una descripción larga y extrae la Unidad y las Tasas que suelen estar al final.
    Ejemplo entrada: "Varillas de acero... Kg 35 25"
    Ejemplo salida: Desc="Varillas de acero...", Unidad="Kg", Tasa1="35", Tasa2="25"
    """
    texto_completo = descripcion_sucia.strip()
    
    # Regex inversa: Busca [Unidad] [Tasa] [Tasa] al final de la línea
    # Soporta: Kg, Pza, L, M2, etc. y tasas numéricas o "Ex" (Exento)
    patron_final = re.compile(r"(Kg|Pza|L|M2|Par|Jgo|Mil|Cabeza)\s+([0-9\.]+|Ex\.?)\s+([0-9\.]+|Ex\.?|=|-)$", re.IGNORECASE)
    
    match = patron_final.search(texto_completo)
    
    item = {
        "CODIGO": codigo,
        "DESCRIPCION": texto_completo, # Por defecto todo es descripción
        "UNIDAD": "",
        "ARANCEL_INICIATIVA": "",
        "ARANCEL_DICTAMEN": ""
    }

    if match:
        item["UNIDAD"] = match.group(1)
        item["ARANCEL_INICIATIVA"] = match.group(2)
        item["ARANCEL_DICTAMEN"] = match.group(3)
        # Cortamos la descripción justo antes de donde empiezan los datos numéricos
        item["DESCRIPCION"] = texto_completo[:match.start()].strip()
    
    # Limpieza extra de caracteres que el OCR suele confundir
    item["DESCRIPCION"] = item["DESCRIPCION"].replace('|', '').replace('_', '')
    
    return item

def extraer_ligie_ocr(pdf_path):
    print(f"--- Iniciando proceso para: {pdf_path} ---")
    print("PASO 1: Convirtiendo PDF a imágenes (esto puede tardar unos minutos)...")
    
    try:
        # Convertimos PDF a imágenes de alta resolución (300 DPI)
        pages = convert_from_path(pdf_path, dpi=300, poppler_path=POPPLER_PATH)
    except Exception as e:
        print(f"\nERROR CRÍTICO: Falló la conversión de PDF a Imagen.")
        print(f"Detalle: {e}")
        return pd.DataFrame()

    data = []
    total_paginas = len(pages)
    print(f"PDF cargado exitosamente. Total de páginas: {total_paginas}")
    print("PASO 2: Leyendo texto de las imágenes (OCR)...")

    # Variables temporales para reconstruir párrafos rotos
    temp_codigo = None
    temp_desc = []

    for i, image in enumerate(pages):
        # Escaneamos la imagen buscando texto en español
        # --psm 6 asume un bloque de texto uniforme (ideal para tablas/listas)
        text = pytesseract.image_to_string(image, lang='spa', config='--psm 6')
        
        lines = text.split('\n')
        
        for line in lines:
            line = line.strip()
            
            # Filtros de limpieza
            if es_linea_basura(line): continue
            
            # Corrección OCR: Puntos decimales leídos como comas
            line = line.replace(',', '.') 

            # Intentamos encontrar un código al inicio de la línea
            match = REGEX_CODIGO.match(line)
            
            if match:
                # ¡ENCONTRAMOS UN NUEVO CÓDIGO!
                
                # 1. Guardamos el registro ANTERIOR (si existe)
                if temp_codigo:
                    full_desc = " ".join(temp_desc).strip()
                    registro_procesado = procesar_registro(temp_codigo, full_desc)
                    data.append(registro_procesado)

                # 2. Iniciamos el NUEVO registro
                temp_codigo = match.group(1)
                # Lo que sobra en la línea después del código es parte de la descripción
                resto_linea = line[len(temp_codigo):].strip()
                temp_desc = [resto_linea]
                
            else:
                # NO ES CÓDIGO: Es continuación de la descripción del registro actual
                if temp_codigo:
                    temp_desc.append(line)
        
        # Reporte de progreso cada 10 páginas
        if (i+1) % 10 == 0 or (i+1) == total_paginas:
            print(f"Procesando página {i+1} de {total_paginas}...")

    # Guardar el último registro pendiente al terminar el documento
    if temp_codigo:
        full_desc = " ".join(temp_desc).strip()
        data.append(procesar_registro(temp_codigo, full_desc))

    return pd.DataFrame(data)

# ==========================================
# 4. EJECUCIÓN PRINCIPAL
# ==========================================
if __name__ == "__main__":
    if os.path.exists(PDF_PATH):
        df_resultado = extraer_ligie_ocr(PDF_PATH)
        
        if not df_resultado.empty:
            print("\nPASO 3: Guardando archivo Excel...")
            # Reordenar columnas para que se vea bonito
            cols = ["CODIGO", "DESCRIPCION", "UNIDAD", "ARANCEL_INICIATIVA", "ARANCEL_DICTAMEN"]
            df_resultado = df_resultado[cols]
            
            df_resultado.to_excel(OUTPUT_EXCEL, index=False)
            print(f"¡ÉXITO TOTAL! Archivo guardado como: {OUTPUT_EXCEL}")
            print(f"Se extrajeron {len(df_resultado)} fracciones arancelarias.")
            print("\nMuestra de los primeros 5 datos:")
            print(df_resultado.head())
        else:
            print("El proceso terminó pero no se encontraron datos válidos.")
    else:
        print(f"No encuentro el archivo PDF: {PDF_PATH}")
        print("Asegúrate de poner este script en la misma carpeta que el PDF.")

--- Iniciando proceso para: C:/Users/Edward/Desktop/Bancomext/Tariffs/data/raw/20251209-V.pdf ---
PASO 1: Convirtiendo PDF a imágenes (esto puede tardar unos minutos)...
PDF cargado exitosamente. Total de páginas: 190
PASO 2: Leyendo texto de las imágenes (OCR)...
Procesando página 10 de 190...
Procesando página 20 de 190...
Procesando página 30 de 190...
Procesando página 40 de 190...
Procesando página 50 de 190...
Procesando página 60 de 190...
Procesando página 70 de 190...
Procesando página 80 de 190...
Procesando página 90 de 190...
Procesando página 100 de 190...
Procesando página 110 de 190...
Procesando página 120 de 190...
Procesando página 130 de 190...
Procesando página 140 de 190...
Procesando página 150 de 190...
Procesando página 160 de 190...
Procesando página 170 de 190...
Procesando página 180 de 190...
Procesando página 190 de 190...

PASO 3: Guardando archivo Excel...
¡ÉXITO TOTAL! Archivo guardado como: LIGIE_Tabla_Final.xlsx
Se extrajeron 2018 fracciones arancelari

In [12]:
import os
from pdf2image import convert_from_path

# --- CONFIGURACIÓN ---
# Ruta a tu PDF
PDF_PATH = "C:/Users/Edward/Desktop/Bancomext/Tariffs/data/raw/20251209-V.pdf"
# Ruta a la carpeta bin de Poppler (Ajústala si es necesario)
POPPLER_PATH = r'C:/Program Files/Poppler/poppler-25.12.0/Library/bin'
# Carpeta donde se guardarán las imágenes
OUTPUT_FOLDER = "paginas_ligie_img"

def convertir_pdf_a_imagenes():
    if not os.path.exists(OUTPUT_FOLDER):
        os.makedirs(OUTPUT_FOLDER)
        print(f"Carpeta creada: {OUTPUT_FOLDER}")

    print(f"Iniciando conversión de {PDF_PATH}...")
    
    try:
        # Convertimos a 300 DPI (Alta calidad para que el OCR lea bien los puntos)
        pages = convert_from_path(PDF_PATH, dpi=300, poppler_path=POPPLER_PATH)
        
        for i, page in enumerate(pages):
            image_name = f"pag_{i+1:04d}.jpg"
            image_path = os.path.join(OUTPUT_FOLDER, image_name)
            page.save(image_path, "JPEG")
            
            if (i+1) % 10 == 0:
                print(f"Guardada {image_name}...")
                
        print(f"¡Listo! {len(pages)} páginas convertidas en la carpeta '{OUTPUT_FOLDER}'.")
        
    except Exception as e:
        print(f"Error: {e}")
        print("Verifica la ruta de Poppler.")

if __name__ == "__main__":
    convertir_pdf_a_imagenes()

Carpeta creada: paginas_ligie_img
Iniciando conversión de C:/Users/Edward/Desktop/Bancomext/Tariffs/data/raw/20251209-V.pdf...
Guardada pag_0010.jpg...
Guardada pag_0020.jpg...
Guardada pag_0030.jpg...
Guardada pag_0040.jpg...
Guardada pag_0050.jpg...
Guardada pag_0060.jpg...
Guardada pag_0070.jpg...
Guardada pag_0080.jpg...
Guardada pag_0090.jpg...
Guardada pag_0100.jpg...
Guardada pag_0110.jpg...
Guardada pag_0120.jpg...
Guardada pag_0130.jpg...
Guardada pag_0140.jpg...
Guardada pag_0150.jpg...
Guardada pag_0160.jpg...
Guardada pag_0170.jpg...
Guardada pag_0180.jpg...
Guardada pag_0190.jpg...
¡Listo! 190 páginas convertidas en la carpeta 'paginas_ligie_img'.


In [13]:
import pytesseract
try:
    from PIL import Image
except ImportError:
    import Image
import pandas as pd
import re
import os
import sys

# ==========================================
# 1. CONFIGURACIÓN
# ==========================================
# Ajusta tu ruta de Tesseract
TESSERACT_CMD = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
# Carpeta donde guardaste las imágenes en el paso anterior
INPUT_FOLDER = "paginas_ligie_img"
OUTPUT_EXCEL = "LIGIE_Base_Datos_Final.xlsx"

# Configurar Tesseract
pytesseract.pytesseract.tesseract_cmd = TESSERACT_CMD

# --- REGLAS DE NEGOCIO ---
# 1. Código: 8 dígitos con puntos
REGEX_INICIO_FILA = re.compile(r"^(\d{4}\.\d{2}\.\d{2})")

# 2. Unidades permitidas (Añade más si faltan)
UNIDADES_VALIDAS = r"(Kg|Pza|L|M2|Par|Jgo|Mil|Cabeza)"

# 3. Cuotas permitidas: Números, decimales, "Ex.", "Ex", o "=" (sin cambios)
# La regex busca: Espacio + Tasa + Espacio + Tasa + Fin de linea
# Se usa re.IGNORECASE para que "kg" o "KG" funcionen igual
REGEX_FINAL_LINEA = re.compile(rf"{UNIDADES_VALIDAS}\s+([0-9\.]+|Ex\.?|=)\s*([0-9\.]+|Ex\.?|=)?$", re.IGNORECASE)

# Lista negra para limpiar basura antes de procesar
TEXTOS_BASURA = [
    "CÁMARA DE", "LXVI LEGISLATURA", "DICTAMEN", "Página", 
    "Comisión de", "INICIATIVA", "EJECUTIVO", "FEDERAL", 
    "CUOTA", "(ARANCEL)", "MODIFICACIÓN", "The following table",
    "Hoja", "Anexo"
]

# ==========================================
# 2. FUNCIONES DE LÓGICA
# ==========================================

def limpiar_texto_ocr(texto):
    """Limpia ruido común del OCR."""
    # Reemplazar pipes o guiones bajos que el OCR confunde con bordes de tabla
    texto = texto.replace('|', ' ').replace('_', ' ')
    # Quitar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto).strip()
    return texto

def es_basura(linea):
    if len(linea) < 3: return True
    for basura in TEXTOS_BASURA:
        if basura.lower() in linea.lower():
            return True
    return False

def procesar_bloque_texto(codigo, texto_acumulado):
    """
    Recibe: "0101.01.01 Caballos vivos reproductores de raza pura. Cabeza Ex. Ex."
    Retorna: Diccionario con campos separados o None si falla.
    """
    # 1. Quitamos el código del inicio para analizar el resto
    contenido = texto_acumulado.replace(codigo, "", 1).strip()
    
    # 2. Buscamos Unidad y Tasas al FINAL del texto (Estrategia Inversa)
    match = REGEX_FINAL_LINEA.search(contenido)
    
    if match:
        unidad = match.group(1)      # Ej: Kg
        tasa_1 = match.group(2)      # Ej: 35
        tasa_2 = match.group(3)      # Ej: 25 (puede ser None)
        
        # Si la tasa 2 está vacía, asumimos que es igual o nula, ajustamos según lógica
        if tasa_2 is None: tasa_2 = "" 

        # 3. La descripción es todo lo que está ANTES de lo que encontró el Regex
        fin_descripcion_idx = match.start()
        descripcion = contenido[:fin_descripcion_idx].strip()
        
        # 4. Regla de validación: Descripción debe terminar en punto
        # Si el OCR falló y puso una coma, lo corregimos. Si no tiene nada, le ponemos punto.
        if descripcion.endswith(','):
            descripcion = descripcion[:-1] + "."
        elif not descripcion.endswith('.'):
            # Opcional: Forzar el punto o marcar error. Aquí lo forzamos para que sea 'usable'
            descripcion += "."

        # Imprimir para monitoreo en tiempo real (Feedback visual)
        print(f"  [OK] {codigo} | Unid: {unidad} | T1: {tasa_1} | T2: {tasa_2} | Desc: {descripcion[:30]}...")
        
        return {
            "CODIGO": codigo,
            "DESCRIPCION": descripcion,
            "UNIDAD": unidad,
            "TASA_INICIATIVA": tasa_1,
            "TASA_DICTAMEN": tasa_2,
            "ESTATUS": "COMPLETO"
        }
    else:
        # Si falla el regex del final, es porque el OCR leyó mal la unidad o las tasas
        print(f"  [ERROR DE FORMATO] No se pudo separar Unidad/Tasas para el código {codigo}")
        print(f"     Texto crudo: {contenido}")
        return {
            "CODIGO": codigo,
            "DESCRIPCION": contenido, # Guardamos todo junto para corrección manual
            "UNIDAD": "REVISAR",
            "TASA_INICIATIVA": "REVISAR",
            "TASA_DICTAMEN": "REVISAR",
            "ESTATUS": "ERROR_OCR"
        }

# ==========================================
# 3. BUCLE PRINCIPAL DE EXTRACCIÓN
# ==========================================

def main_extraction():
    if not os.path.exists(INPUT_FOLDER):
        print("Error: No existe la carpeta de imágenes. Ejecuta el Script 1 primero.")
        return

    archivos = sorted(os.listdir(INPUT_FOLDER))
    # Filtrar solo jpgs
    archivos = [f for f in archivos if f.endswith(".jpg")]
    
    registros_finales = []
    
    # Variables de estado para la máquina de estados
    buffer_codigo = None
    buffer_texto = []

    print(f"Iniciando extracción sobre {len(archivos)} imágenes...")

    for i, archivo in enumerate(archivos):
        ruta_img = os.path.join(INPUT_FOLDER, archivo)
        
        # OCR de la página
        # --psm 6 asume un bloque de texto uniforme (clave para tablas sin bordes)
        texto_crudo = pytesseract.image_to_string(Image.open(ruta_img), lang='spa', config='--psm 6')
        
        lineas = texto_crudo.split('\n')
        
        print(f"--> Procesando {archivo} ({len(lineas)} líneas detectadas)")

        for linea in lineas:
            linea_limpia = limpiar_texto_ocr(linea)
            
            if es_basura(linea_limpia):
                continue

            # ¿Empieza con un código arancelario?
            match_codigo = REGEX_INICIO_FILA.match(linea_limpia)

            if match_codigo:
                # 1. Si ya teníamos un buffer lleno, procesamos el registro ANTERIOR
                if buffer_codigo:
                    texto_completo_anterior = " ".join(buffer_texto)
                    fila = procesar_bloque_texto(buffer_codigo, texto_completo_anterior)
                    if fila: registros_finales.append(fila)

                # 2. Iniciamos NUEVO registro
                buffer_codigo = match_codigo.group(1)
                buffer_texto = [linea_limpia] # Guardamos la línea entera, luego limpiaremos el código
            
            else:
                # 3. Si no es código, es continuación de la descripción del actual
                if buffer_codigo:
                    buffer_texto.append(linea_limpia)

    # Procesar el último registro pendiente al finalizar los archivos
    if buffer_codigo:
        texto_completo_anterior = " ".join(buffer_texto)
        fila = procesar_bloque_texto(buffer_codigo, texto_completo_anterior)
        registros_finales.append(fila)

    # ==========================================
    # 4. EXPORTACIÓN
    # ==========================================
    if registros_finales:
        df = pd.DataFrame(registros_finales)
        
        # Filtro final de calidad: No puede haber vacíos en columnas críticas
        # (Aunque mi lógica rellena con "REVISAR", aquí podrías filtrar esos si quisieras)
        
        df.to_excel(OUTPUT_EXCEL, index=False)
        print("\n" + "="*40)
        print(f"PROCESO TERMINADO. Datos guardados en: {OUTPUT_EXCEL}")
        print(f"Registros extraídos: {len(df)}")
        print(f"Registros con error de formato: {len(df[df['ESTATUS'] == 'ERROR_OCR'])}")
        print("="*40)
    else:
        print("No se extrajo ningún registro. Revisa la configuración de Tesseract.")

if __name__ == "__main__":
    main_extraction()

Iniciando extracción sobre 190 imágenes...
--> Procesando pag_0001.jpg (14 líneas detectadas)
--> Procesando pag_0002.jpg (38 líneas detectadas)
--> Procesando pag_0003.jpg (38 líneas detectadas)
--> Procesando pag_0004.jpg (47 líneas detectadas)
--> Procesando pag_0005.jpg (39 líneas detectadas)
--> Procesando pag_0006.jpg (42 líneas detectadas)
--> Procesando pag_0007.jpg (36 líneas detectadas)
--> Procesando pag_0008.jpg (30 líneas detectadas)
--> Procesando pag_0009.jpg (28 líneas detectadas)
--> Procesando pag_0010.jpg (30 líneas detectadas)
--> Procesando pag_0011.jpg (36 líneas detectadas)
--> Procesando pag_0012.jpg (32 líneas detectadas)
--> Procesando pag_0013.jpg (1 líneas detectadas)
--> Procesando pag_0014.jpg (32 líneas detectadas)
--> Procesando pag_0015.jpg (37 líneas detectadas)
--> Procesando pag_0016.jpg (35 líneas detectadas)
--> Procesando pag_0017.jpg (43 líneas detectadas)
--> Procesando pag_0018.jpg (44 líneas detectadas)
--> Procesando pag_0019.jpg (42 líneas d

In [15]:
import cv2
import numpy as np
from PIL import Image
import pytesseract
import pandas as pd
import re
import os

# ==========================================
# CONFIGURACIÓN
# ==========================================
TESSERACT_CMD = r'C:/Program Files/Tesseract-OCR/tesseract.exe'
INPUT_FOLDER = "paginas_ligie_img"
OUTPUT_EXCEL = "LIGIE_Extraido_OCR_Columnas.xlsx"

pytesseract.pytesseract.tesseract_cmd = TESSERACT_CMD

# ==========================================
# 1. PREPROCESAMIENTO DE IMAGEN
# ==========================================

def preprocesar_imagen_tabla(img_path, debug=False):
    """
    Optimiza la imagen específicamente para tablas con líneas tenues
    """
    # Leer imagen
    img = cv2.imread(img_path)
    
    # Convertir a escala de grises
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Aumentar contraste agresivamente
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    contrast = clahe.apply(gray)
    
    # Binarización adaptativa (crucial para tablas con fondo gris)
    binary = cv2.adaptiveThreshold(
        contrast, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY,
        15, 10
    )
    
    # Invertir si el fondo es oscuro
    if np.mean(binary) < 127:
        binary = cv2.bitwise_not(binary)
    
    # Eliminar ruido pequeño
    kernel = np.ones((2,2), np.uint8)
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    
    # Guardar imagen debug si se solicita
    if debug:
        debug_path = img_path.replace('.jpg', '_debug.jpg')
        cv2.imwrite(debug_path, cleaned)
        print(f"  [DEBUG] Imagen procesada guardada en: {debug_path}")
    
    return Image.fromarray(cleaned)

# ==========================================
# 2. OCR CON COORDENADAS (CLAVE)
# ==========================================

def extraer_texto_con_coordenadas(img_path):
    """
    Usa pytesseract.image_to_data() para obtener coordenadas de cada palabra.
    Esto permite reconstruir columnas espacialmente.
    """
    img_prep = preprocesar_imagen_tabla(img_path)
    
    # Configuración optimizada para tablas
    custom_config = r'--oem 3 --psm 6 -c preserve_interword_spaces=1'
    
    # Extraer con coordenadas
    data = pytesseract.image_to_data(
        img_prep,
        lang='spa',
        config=custom_config,
        output_type=pytesseract.Output.DICT
    )
    
    return data

# ==========================================
# 3. RECONSTRUCCIÓN DE COLUMNAS
# ==========================================

def agrupar_por_filas(data, tolerancia_y=15):
    """
    Agrupa palabras que están en la misma línea horizontal (misma fila de tabla).
    tolerancia_y: píxeles de diferencia permitidos en el eje Y
    """
    palabras = []
    
    n_boxes = len(data['text'])
    for i in range(n_boxes):
        # Filtrar palabras vacías y con baja confianza
        if int(data['conf'][i]) > 30 and data['text'][i].strip():
            palabras.append({
                'texto': data['text'][i].strip(),
                'x': data['left'][i],
                'y': data['top'][i],
                'w': data['width'][i],
                'h': data['height'][i],
                'conf': data['conf'][i]
            })
    
    # Ordenar por posición vertical (Y)
    palabras.sort(key=lambda p: p['y'])
    
    # Agrupar en filas
    filas = []
    fila_actual = []
    y_ref = None
    
    for palabra in palabras:
        if y_ref is None:
            y_ref = palabra['y']
            fila_actual.append(palabra)
        elif abs(palabra['y'] - y_ref) <= tolerancia_y:
            # Misma fila
            fila_actual.append(palabra)
        else:
            # Nueva fila
            if fila_actual:
                filas.append(fila_actual)
            fila_actual = [palabra]
            y_ref = palabra['y']
    
    # Agregar última fila
    if fila_actual:
        filas.append(fila_actual)
    
    return filas

def detectar_columnas(filas):
    """
    Detecta las posiciones X de las columnas analizando múltiples filas.
    Retorna los rangos X de cada columna.
    """
    # Recolectar todas las posiciones X iniciales
    x_positions = []
    for fila in filas[:20]:  # Analizar primeras 20 filas
        for palabra in fila:
            x_positions.append(palabra['x'])
    
    # Clusterizar posiciones X (columnas típicas)
    x_positions.sort()
    
    columnas = []
    tolerancia_x = 50  # Píxeles de tolerancia horizontal
    
    for x in x_positions:
        # ¿Esta posición es una nueva columna o pertenece a una existente?
        agregado = False
        for col in columnas:
            if abs(x - col['x_inicio']) <= tolerancia_x:
                agregado = True
                break
        
        if not agregado:
            columnas.append({'x_inicio': x, 'x_fin': x + 200})  # Ancho estimado
    
    # Ordenar columnas de izquierda a derecha
    columnas.sort(key=lambda c: c['x_inicio'])
    
    # Ajustar límites finales
    for i in range(len(columnas) - 1):
        columnas[i]['x_fin'] = columnas[i+1]['x_inicio'] - 5
    
    return columnas

def asignar_palabras_a_columnas(fila, columnas):
    """
    Asigna cada palabra de una fila a su columna correspondiente.
    """
    celdas = [[] for _ in columnas]
    
    for palabra in fila:
        x_centro = palabra['x'] + palabra['w'] / 2
        
        for idx, col in enumerate(columnas):
            if col['x_inicio'] <= x_centro <= col['x_fin']:
                celdas[idx].append(palabra['texto'])
                break
    
    # Unir palabras dentro de cada celda
    return [' '.join(celda) for celda in celdas]

# ==========================================
# 4. VALIDACIÓN Y LIMPIEZA
# ==========================================

def validar_fila_arancelaria(fila_texto):
    """
    Valida que una fila extraída contenga un código arancelario válido.
    """
    if not fila_texto or len(fila_texto) < 3:
        return False
    
    codigo = fila_texto[0]
    
    # Regex para código arancelario
    if re.match(r'\d{4}\.\d{2}\.\d{2}', codigo):
        return True
    
    return False

def limpiar_descripcion(texto):
    """
    Limpia errores comunes del OCR en descripciones.
    """
    # Eliminar caracteres extraños
    texto = re.sub(r'[|_]', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto).strip()
    
    # Asegurar punto final
    if texto and not texto.endswith('.'):
        texto += '.'
    
    return texto

# ==========================================
# 5. PROCESO PRINCIPAL
# ==========================================

def procesar_imagenes_con_columnas():
    """
    Proceso completo: lee imágenes, extrae con coordenadas, reconstruye tabla.
    """
    if not os.path.exists(INPUT_FOLDER):
        print(f"Error: No existe la carpeta {INPUT_FOLDER}")
        return
    
    archivos = sorted([f for f in os.listdir(INPUT_FOLDER) if f.endswith('.jpg')])
    
    registros_finales = []
    
    print(f"Iniciando extracción inteligente sobre {len(archivos)} imágenes...\n")
    
    for idx, archivo in enumerate(archivos, 1):
        img_path = os.path.join(INPUT_FOLDER, archivo)
        
        print(f"[{idx}/{len(archivos)}] Procesando: {archivo}")
        
        try:
            # 1. OCR con coordenadas
            data = extraer_texto_con_coordenadas(img_path)
            
            # 2. Agrupar en filas
            filas = agrupar_por_filas(data, tolerancia_y=15)
            
            if not filas:
                print(f"  ⚠️  No se detectó texto en la página")
                continue
            
            # 3. Detectar columnas (solo en primera página o cada 10 páginas)
            if idx == 1 or idx % 10 == 0:
                columnas = detectar_columnas(filas)
                print(f"  ✓ Detectadas {len(columnas)} columnas")
            
            # 4. Procesar cada fila
            for fila in filas:
                celdas = asignar_palabras_a_columnas(fila, columnas)
                
                # Validar que sea una fila de datos
                if validar_fila_arancelaria(celdas):
                    
                    registro = {
                        'CODIGO': celdas[0] if len(celdas) > 0 else '',
                        'DESCRIPCION': limpiar_descripcion(celdas[1]) if len(celdas) > 1 else '',
                        'UNIDAD': celdas[2] if len(celdas) > 2 else '',
                        'TASA_INICIATIVA': celdas[3] if len(celdas) > 3 else '',
                        'TASA_DICTAMEN': celdas[4] if len(celdas) > 4 else '',
                        'PAGINA': idx,
                        'ARCHIVO': archivo
                    }
                    
                    registros_finales.append(registro)
                    print(f"  ✓ {registro['CODIGO'][:15]}... | {registro['UNIDAD']}")
        
        except Exception as e:
            print(f"  ❌ Error en {archivo}: {str(e)}")
            continue
    
    # ==========================================
    # 6. EXPORTACIÓN
    # ==========================================
    
    if registros_finales:
        df = pd.DataFrame(registros_finales)
        
        # Estadísticas
        total = len(df)
        completos = len(df[df['DESCRIPCION'].str.len() > 10])
        
        df.to_excel(OUTPUT_EXCEL, index=False)
        
        print("\n" + "="*60)
        print(f"✅ EXTRACCIÓN COMPLETADA")
        print(f"📁 Archivo: {OUTPUT_EXCEL}")
        print(f"📊 Total registros: {total}")
        print(f"✓ Registros completos: {completos} ({completos/total*100:.1f}%)")
        print("="*60)
        
        # Mostrar muestra
        print("\n🔍 MUESTRA DE DATOS:")
        print(df.head(10).to_string())
        
    else:
        print("\n❌ No se extrajo ningún registro. Revisa las imágenes.")

if __name__ == "__main__":
    # Opción: Generar imagen debug de la primera página
    # preprocesar_imagen_tabla(os.path.join(INPUT_FOLDER, "pag_0001.jpg"), debug=True)
    
    procesar_imagenes_con_columnas()

Iniciando extracción inteligente sobre 190 imágenes...

[1/190] Procesando: pag_0001.jpg
  ✓ Detectadas 25 columnas
[2/190] Procesando: pag_0002.jpg
[3/190] Procesando: pag_0003.jpg
[4/190] Procesando: pag_0004.jpg
[5/190] Procesando: pag_0005.jpg
[6/190] Procesando: pag_0006.jpg
[7/190] Procesando: pag_0007.jpg
[8/190] Procesando: pag_0008.jpg
[9/190] Procesando: pag_0009.jpg
[10/190] Procesando: pag_0010.jpg
  ✓ Detectadas 28 columnas
[11/190] Procesando: pag_0011.jpg
[12/190] Procesando: pag_0012.jpg
[13/190] Procesando: pag_0013.jpg
  ⚠️  No se detectó texto en la página
[14/190] Procesando: pag_0014.jpg
[15/190] Procesando: pag_0015.jpg
[16/190] Procesando: pag_0016.jpg
[17/190] Procesando: pag_0017.jpg
[18/190] Procesando: pag_0018.jpg
[19/190] Procesando: pag_0019.jpg
[20/190] Procesando: pag_0020.jpg
  ✓ Detectadas 31 columnas
[21/190] Procesando: pag_0021.jpg
[22/190] Procesando: pag_0022.jpg
[23/190] Procesando: pag_0023.jpg
[24/190] Procesando: pag_0024.jpg
[25/190] Procesan

In [1]:
pip install img2table --user

  Using cached img2table-1.4.2-py3-none-any.whl.metadata (21 kB)
  Using cached polars-1.36.0-py3-none-any.whl.metadata (10 kB)
  Using cached opencv_contrib_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached numba-0.63.0-cp313-cp313-win_amd64.whl.metadata (3.0 kB)
  Using cached beautifulsoup4-4.14.3-py3-none-any.whl.metadata (3.8 kB)
  Using cached llvmlite-0.46.0-cp313-cp313-win_amd64.whl.metadata (5.1 kB)
Using cached img2table-1.4.2-py3-none-any.whl (91 kB)
Using cached opencv_contrib_python_headless-4.12.0.88-cp37-abi3-win_amd64.whl (45.2 MB)
Using cached polars-1.36.0-py3-none-any.whl (801 kB)
Using cached beautifulsoup4-4.14.3-py3-none-any.whl (107 kB)
Using cached numba-0.63.0-cp313-cp313-win_amd64.whl (2.8 MB)
Using cached llvmlite-0.46.0-cp313-cp313-win_amd64.whl (38.1 MB)

   ---------------------------------------- 0/6 [polars]
   ---------------------------------------- 0/6 [polars]
   ---------------------------------------- 0/6 [polars]
  

In [19]:
import pandas as pd
from img2table.document import PDF
from img2table.ocr import TesseractOCR
import os
import re

# ==========================================
# 1. CONFIGURACIÓN DE RUTAS
# ==========================================
# Ajusta estas rutas a tu entorno local
RUTA_TESSERACT = r"C:/Program Files/Tesseract-OCR"
ARCHIVO_PDF = r"C:/Users/Edward/Downloads/20251209-V-30-109.pdf"
ARCHIVO_SALIDA = "tablas_extraidas_corregido.xlsx"

# Nombres estándar para las columnas (para evitar desfases)
COLUMNAS_ESTANDAR = [
    "CÓDIGO", 
    "DESCRIPCIÓN", 
    "UNIDAD", 
    "CUOTA_INICIATIVA", 
    "CUOTA_DICTAMEN"
]

# ==========================================
# 2. FUNCIONES DE LIMPIEZA
# ==========================================

def configurar_entorno():
    if os.path.exists(RUTA_TESSERACT):
        os.environ["PATH"] += os.pathsep + RUTA_TESSERACT
        return True
    else:
        print(f"❌ ERROR: No se encuentra Tesseract en: {RUTA_TESSERACT}")
        return False

def limpiar_texto(val):
    """Limpia saltos de línea y espacios extra."""
    if pd.isna(val):
        return val
    return str(val).replace('\n', ' ').strip()

def intentar_separar_cuotas(row):
    """
    Si la columna de Dictamen está vacía pero la de Iniciativa tiene dos números
    (ej: '35 25'), intenta separarlos.
    """
    iniciativa = str(row['CUOTA_INICIATIVA']).strip()
    dictamen = str(row['CUOTA_DICTAMEN']).strip()
    
    # Si dictamen está vacío (o es nan/None) y la iniciativa parece tener dos valores
    if (dictamen == 'nan' or dictamen == 'None' or dictamen == '') and ' ' in iniciativa:
        partes = iniciativa.split()
        # Verificamos si parece que son dos números (ej. '10 7' o '35 25')
        if len(partes) >= 2:
            # Asumimos que el último valor es el dictamen
            nuevo_dictamen = partes[-1]
            nueva_iniciativa = " ".join(partes[:-1])
            return nueva_iniciativa, nuevo_dictamen
            
    return row['CUOTA_INICIATIVA'], row['CUOTA_DICTAMEN']

def estandarizar_tabla(df, num_pagina):
    """
    Limpia y estandariza una tabla individual antes de concatenarla.
    """
    # 1. Eliminar filas vacías
    df = df.dropna(how='all')
    
    # 2. Eliminar columnas que sean totalmente vacías (bordes fantasma)
    df = df.dropna(axis=1, how='all')
    
    # 3. Normalizar a 5 columnas
    # Si hay más de 5, conservamos las 5 primeras (asumiendo orden izquierda-derecha)
    # Si hay menos, añadimos vacías.
    columnas_actuales = df.shape[1]
    
    if columnas_actuales > 5:
        # A veces crea columnas extra al final vacías o con basura
        df = df.iloc[:, :5]
    elif columnas_actuales < 5:
        # Rellenar con columnas vacías hasta llegar a 5
        for _ in range(5 - columnas_actuales):
            df[len(df.columns)] = None

    # 4. Asignar nombres estándar
    df.columns = COLUMNAS_ESTANDAR
    
    # 5. Detectar y eliminar fila de encabezado si existe en los datos
    # Buscamos si la primera fila contiene "CÓDIGO" o "DESCRIPCIÓN"
    if not df.empty:
        fila_0 = " ".join([str(x).upper() for x in df.iloc[0].values])
        if "CÓDIGO" in fila_0 or "DESCRIPCIÓN" in fila_0:
            df = df.iloc[1:] # Eliminar primera fila
    
    # 6. Añadir metadato de página
    df['Pagina_Origen'] = num_pagina + 1
    
    return df

# ==========================================
# 3. EXTRACCIÓN PRINCIPAL
# ==========================================

def extraer_tablas(ruta_pdf):
    try:
        ocr = TesseractOCR(n_threads=1, lang="spa")
    except Exception as e:
        print(f"❌ Error OCR: {e}")
        return None

    print(f"🔄 Leyendo PDF: {ruta_pdf}...")
    doc = PDF(src=ruta_pdf)

    print("⏳ Ejecutando OCR (esto puede tardar)...")
    try:
        # borderless_tables=False para usar las líneas de la tabla
        tablas_extraidas = doc.extract_tables(ocr=ocr,
                                              implicit_rows=False,
                                              borderless_tables=False,
                                              min_confidence=50)
    except Exception as e:
        print(f"❌ Error extrayendo tablas: {e}")
        return None

    lista_dfs = []

    for numero_pagina, tablas in tablas_extraidas.items():
        for tabla in tablas:
            df = tabla.df
            
            # Procesar la tabla individualmente
            df_limpio = estandarizar_tabla(df, numero_pagina)
            
            # Solo añadir si tiene datos
            if not df_limpio.empty:
                lista_dfs.append(df_limpio)

    if lista_dfs:
        print(f"✅ Se encontraron {len(lista_dfs)} tablas. Concatenando...")
        
        # Concatenación segura (ahora todos tienen las mismas columnas)
        df_final = pd.concat(lista_dfs, ignore_index=True)
        
        # ==========================================
        # 4. LIMPIEZA POST-CONCATENACIÓN
        # ==========================================
        
        # A. Limpieza de texto general
        for col in COLUMNAS_ESTANDAR:
            df_final[col] = df_final[col].apply(limpiar_texto)
            
        # B. Eliminar filas que sean encabezados repetidos (basura residual)
        # Filtramos filas donde la columna CÓDIGO diga literalmente "CÓDIGO" o similar
        filtro_basura = df_final['CÓDIGO'].str.upper().str.contains("CÓDIGO|DESCRIPCIÓN", na=False)
        df_final = df_final[~filtro_basura]
        
        # C. Eliminar filas donde el CÓDIGO sea demasiado corto (ruido OCR)
        df_final = df_final[df_final['CÓDIGO'].str.len() > 3]
        
        # D. Corregir "Huecos" en las cuotas (Separar valores fusionados)
        # Aplica la lógica fila por fila
        df_final[['CUOTA_INICIATIVA', 'CUOTA_DICTAMEN']] = df_final.apply(
            lambda x: pd.Series(intentar_separar_cuotas(x)), axis=1
        )

        return df_final
    else:
        return None

# ==========================================
# 4. EJECUCIÓN
# ==========================================

if __name__ == "__main__":
    if configurar_entorno():
        if os.path.exists(ARCHIVO_PDF):
            df_resultado = extraer_tablas(ARCHIVO_PDF)
            
            if df_resultado is not None and not df_resultado.empty:
                try:
                    df_resultado.to_excel(ARCHIVO_SALIDA, index=False)
                    print(f"🎉 ¡ÉXITO! Archivo guardado: {ARCHIVO_SALIDA}")
                    print(df_resultado.head())
                    print(f"\nTotal de filas extraídas: {len(df_resultado)}")
                except PermissionError:
                    print(f"❌ ERROR: Cierra el archivo Excel '{ARCHIVO_SALIDA}' antes de correr el código.")
            else:
                print("⚠️ No se pudo extraer información válida.")
        else:
            print(f"❌ No encuentro el archivo: {ARCHIVO_PDF}")

🔄 Leyendo PDF: C:/Users/Edward/Downloads/20251209-V-30-109.pdf...
⏳ Ejecutando OCR (esto puede tardar)...
✅ Se encontraron 78 tablas. Concatenando...
🎉 ¡ÉXITO! Archivo guardado: tablas_extraidas_corregido.xlsx
       CÓDIGO                                     DESCRIPCIÓN UNIDAD  \
1  3303.00.01                               Aguas de tocador.   None   
2  3303.00.99                                      Los demás.   None   
3   3304.10.0  Preparaciones para el maquillaje de os labios.     Kg   
4   3304.20.0   Preparaciones para el maquillaje de los ojos.     Kg   
5   3304.30.0       Preparaciones para manicuras o pedicuros.     Kg   

  CUOTA_INICIATIVA CUOTA_DICTAMEN  Pagina_Origen  
1               35             25              1  
2               35             25              1  
3               50             36              1  
4               35             25              1  
5               35             25              1  

Total de filas extraídas: 1438


In [6]:
pip install pdf2image pytesseract opencv-python pandas pillow numpy

Note: you may need to restart the kernel to use updated packages.


In [13]:
"""
Extractor de tablas desde PDF con imágenes usando OCR
Requiere: pip install PyMuPDF pytesseract opencv-python pandas pillow numpy
También necesitas instalar Tesseract OCR en tu sistema:
- Windows: https://github.com/UB-Mannheim/tesseract/wiki
- Linux: sudo apt-get install tesseract-ocr tesseract-ocr-spa
- Mac: brew install tesseract tesseract-lang
"""

import cv2
import numpy as np
import pytesseract
import fitz  # PyMuPDF
from PIL import Image
import pandas as pd
import re
import os

# ============================================================================
# CONFIGURACIÓN DE RUTAS - Ajusta según tu instalación
# ============================================================================

# Ruta de Tesseract OCR (necesario para el OCR)
# Descomenta y ajusta según tu instalación:
pytesseract.pytesseract.tesseract_cmd = r'C:/Program Files/Tesseract-OCR/tesseract.exe'

# Ruta de Poppler (opcional, solo si usas pdf2image en lugar de PyMuPDF)
# Ejemplo: POPPLER_PATH = r'C:\poppler-24.08.0\Library\bin'
POPPLER_PATH = r'C:/Program Files/Poppler/poppler-25.12.0/Library/bin'

# ============================================================================

class PDFTableExtractor:
    def __init__(self, pdf_path, idioma='spa', usar_pymupdf=True):
        """
        Inicializa el extractor de tablas
        
        Args:
            pdf_path: Ruta al archivo PDF
            idioma: Idioma para OCR ('spa' para español, 'eng' para inglés)
            usar_pymupdf: Si True usa PyMuPDF, si False usa pdf2image+Poppler
        """
        self.pdf_path = pdf_path
        self.idioma = idioma
        self.usar_pymupdf = usar_pymupdf
        self.doc = None
        self.images = None
        
    def abrir_pdf(self):
        """Abre el documento PDF"""
        if not os.path.exists(self.pdf_path):
            raise FileNotFoundError(f"No se encuentra el archivo: {self.pdf_path}")
        
        if self.usar_pymupdf:
            self.doc = fitz.open(self.pdf_path)
            print(f"PDF abierto con PyMuPDF: {len(self.doc)} páginas")
        else:
            # Usar pdf2image con Poppler
            from pdf2image import convert_from_path
            print(f"Convirtiendo PDF con Poppler...")
            self.images = convert_from_path(
                self.pdf_path, 
                dpi=300,
                poppler_path=POPPLER_PATH
            )
            print(f"PDF convertido: {len(self.images)} páginas")
        
        return self.doc if self.usar_pymupdf else self.images
    
    def convertir_pagina_a_imagen(self, num_pagina, zoom=3.0):
        """Convierte una página del PDF a imagen de alta resolución"""
        if self.usar_pymupdf:
            # Usar PyMuPDF
            pagina = self.doc[num_pagina]
            
            # Matriz de transformación para aumentar resolución
            mat = fitz.Matrix(zoom, zoom)
            pix = pagina.get_pixmap(matrix=mat)
            
            # Convertir a formato numpy array
            img_data = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
            
            # Si tiene canal alpha, convertir a RGB
            if pix.n == 4:
                img_data = cv2.cvtColor(img_data, cv2.COLOR_RGBA2RGB)
            
            return img_data
        else:
            # Usar pdf2image (ya convertido en abrir_pdf)
            if self.images is None:
                self.abrir_pdf()
            
            # Convertir PIL Image a numpy array
            img_pil = self.images[num_pagina]
            img_data = np.array(img_pil)
            
            # Asegurar que está en RGB
            if len(img_data.shape) == 2:
                img_data = cv2.cvtColor(img_data, cv2.COLOR_GRAY2RGB)
            elif img_data.shape[2] == 4:
                img_data = cv2.cvtColor(img_data, cv2.COLOR_RGBA2RGB)
            
            return img_data
    
    def preprocesar_imagen(self, imagen):
        """Mejora la imagen para mejor OCR"""
        # Convertir a escala de grises
        if len(imagen.shape) == 3:
            gray = cv2.cvtColor(imagen, cv2.COLOR_RGB2GRAY)
        else:
            gray = imagen
        
        # Aplicar umbralización adaptativa
        thresh = cv2.adaptiveThreshold(
            gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
            cv2.THRESH_BINARY, 11, 2
        )
        
        # Reducir ruido
        denoised = cv2.fastNlMeansDenoising(thresh, None, 10, 7, 21)
        
        # Opcional: dilatar ligeramente para unir texto fragmentado
        kernel = np.ones((1, 1), np.uint8)
        dilated = cv2.dilate(denoised, kernel, iterations=1)
        
        return dilated
    
    def detectar_tablas(self, imagen_procesada):
        """Detecta las regiones de tablas en la imagen"""
        # Detectar líneas horizontales y verticales
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
        vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
        
        horizontal_lines = cv2.morphologyEx(imagen_procesada, cv2.MORPH_OPEN, horizontal_kernel)
        vertical_lines = cv2.morphologyEx(imagen_procesada, cv2.MORPH_OPEN, vertical_kernel)
        
        # Combinar líneas
        table_structure = cv2.addWeighted(horizontal_lines, 0.5, vertical_lines, 0.5, 0.0)
        
        # Encontrar contornos (tablas)
        contours, _ = cv2.findContours(table_structure, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Filtrar contornos por área mínima
        table_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 5000]
        
        # Ordenar contornos de arriba hacia abajo
        table_contours = sorted(table_contours, key=lambda cnt: cv2.boundingRect(cnt)[1])
        
        return table_contours
    
    def extraer_texto_ocr(self, imagen, config_personalizado=''):
        """Extrae texto usando Tesseract OCR"""
        config_ocr = f'--oem 3 --psm 6 -l {self.idioma} {config_personalizado}'
        
        # Convertir numpy array a PIL Image si es necesario
        if isinstance(imagen, np.ndarray):
            imagen = Image.fromarray(imagen)
        
        texto = pytesseract.image_to_string(imagen, config=config_ocr)
        return texto
    
    def extraer_datos_estructurados(self, imagen, region=None):
        """Extrae datos de tabla usando OCR con detección de estructura"""
        if region is not None:
            x, y, w, h = region
            imagen_recortada = imagen[y:y+h, x:x+w]
        else:
            imagen_recortada = imagen
        
        # Convertir a PIL Image
        if isinstance(imagen_recortada, np.ndarray):
            pil_img = Image.fromarray(imagen_recortada)
        else:
            pil_img = imagen_recortada
        
        # Obtener datos con información de posición
        datos_ocr = pytesseract.image_to_data(
            pil_img, 
            lang=self.idioma,
            output_type=pytesseract.Output.DICT
        )
        
        # Organizar texto por líneas basándose en coordenadas Y
        lineas = {}
        for i, texto in enumerate(datos_ocr['text']):
            if texto.strip():
                conf = int(datos_ocr['conf'][i])
                if conf > 20:  # Filtrar baja confianza
                    y_pos = datos_ocr['top'][i]
                    x_pos = datos_ocr['left'][i]
                    
                    # Agrupar por línea (tolerancia de 15 píxeles)
                    linea_encontrada = False
                    for linea_y in lineas.keys():
                        if abs(y_pos - linea_y) < 15:
                            lineas[linea_y].append((x_pos, texto))
                            linea_encontrada = True
                            break
                    
                    if not linea_encontrada:
                        lineas[y_pos] = [(x_pos, texto)]
        
        # Ordenar líneas por posición Y y luego por X
        filas = []
        for y_pos in sorted(lineas.keys()):
            elementos_ordenados = sorted(lineas[y_pos], key=lambda x: x[0])
            fila = ' '.join([elem[1] for elem in elementos_ordenados])
            filas.append(fila)
        
        return filas
    
    def parsear_a_dataframe(self, filas):
        """Convierte las filas extraídas en un DataFrame"""
        if not filas:
            return pd.DataFrame()
        
        # Intentar detectar el encabezado (primera fila no vacía)
        datos = []
        encabezado = None
        
        for fila in filas:
            if fila.strip():
                # Dividir por múltiples espacios (2 o más)
                columnas = re.split(r'\s{2,}', fila.strip())
                if encabezado is None:
                    encabezado = columnas
                else:
                    datos.append(columnas)
        
        if encabezado and datos:
            # Asegurar que todas las filas tengan el mismo número de columnas
            max_cols = max(len(encabezado), max(len(fila) for fila in datos) if datos else 0)
            
            # Rellenar con vacíos si es necesario
            encabezado.extend([''] * (max_cols - len(encabezado)))
            for fila in datos:
                fila.extend([''] * (max_cols - len(fila)))
            
            # Crear DataFrame
            num_cols = len(datos[0]) if datos else len(encabezado)
            df = pd.DataFrame(datos, columns=encabezado[:num_cols])
            return df
        
        return pd.DataFrame()
    
    def procesar_pdf_completo(self, guardar_imagenes=False, zoom=3.0):
        """Procesa todo el PDF y extrae todas las tablas"""
        if self.usar_pymupdf:
            if self.doc is None:
                self.abrir_pdf()
            num_paginas = len(self.doc)
        else:
            if self.images is None:
                self.abrir_pdf()
            num_paginas = len(self.images)
        
        todas_las_tablas = []
        
        for num_pagina in range(num_paginas):
            print(f"\n{'='*60}")
            print(f"Procesando página {num_pagina + 1} de {num_paginas}...")
            print('='*60)
            
            # Convertir página a imagen
            imagen = self.convertir_pagina_a_imagen(num_pagina, zoom=zoom)
            
            # Preprocesar imagen
            img_procesada = self.preprocesar_imagen(imagen)
            
            if guardar_imagenes:
                cv2.imwrite(f'pagina_{num_pagina+1}_original.png', imagen)
                cv2.imwrite(f'pagina_{num_pagina+1}_procesada.png', img_procesada)
                print(f"✓ Imágenes guardadas")
            
            # Detectar tablas
            contornos_tablas = self.detectar_tablas(img_procesada)
            
            if contornos_tablas:
                print(f"✓ Se encontraron {len(contornos_tablas)} tabla(s)")
                
                for i, contorno in enumerate(contornos_tablas):
                    x, y, w, h = cv2.boundingRect(contorno)
                    print(f"\n  → Extrayendo tabla {i+1} (posición: x={x}, y={y}, w={w}, h={h})...")
                    
                    # Extraer datos de esta región
                    filas = self.extraer_datos_estructurados(img_procesada, (x, y, w, h))
                    
                    print(f"    • Filas detectadas: {len(filas)}")
                    
                    if filas:
                        df = self.parsear_a_dataframe(filas)
                        if not df.empty:
                            print(f"    • DataFrame creado: {df.shape[0]} filas x {df.shape[1]} columnas")
                            todas_las_tablas.append({
                                'pagina': num_pagina + 1,
                                'tabla': i + 1,
                                'dataframe': df,
                                'region': (x, y, w, h)
                            })
                        else:
                            print(f"    ⚠ No se pudo crear DataFrame")
            else:
                # Si no se detectan tablas, extraer todo el texto de la página
                print("⚠ No se detectaron tablas estructuradas, extrayendo todo el texto...")
                filas = self.extraer_datos_estructurados(img_procesada)
                
                print(f"  • Filas detectadas: {len(filas)}")
                
                if filas:
                    df = self.parsear_a_dataframe(filas)
                    if not df.empty:
                        print(f"  • DataFrame creado: {df.shape[0]} filas x {df.shape[1]} columnas")
                        todas_las_tablas.append({
                            'pagina': num_pagina + 1,
                            'tabla': 1,
                            'dataframe': df,
                            'region': None
                        })
        
        return todas_las_tablas
    
    def cerrar(self):
        """Cierra el documento PDF"""
        if self.doc:
            self.doc.close()

# Ejemplo de uso
if __name__ == "__main__":
    print("="*60)
    print("EXTRACTOR DE TABLAS PDF CON OCR")
    print("="*60)
    
    # Verificar que Tesseract está instalado
    try:
        version = pytesseract.get_tesseract_version()
        print(f"\n✓ Tesseract versión: {version}")
    except Exception as e:
        print(f"\n✗ ERROR: Tesseract no está instalado o no está en PATH")
        print(f"  Instala desde: https://github.com/UB-Mannheim/tesseract/wiki")
        print(f"  Detalle: {e}")
        exit(1)
    
    # Ruta del PDF
    pdf_path = r'C:/Users/Edward/Downloads/20251209-V-31-33.pdf'
    
    if not os.path.exists(pdf_path):
        print(f"\n✗ ERROR: No se encuentra el archivo {pdf_path}")
        exit(1)
    
    print(f"\n✓ Archivo encontrado: {pdf_path}")
    
    # Elegir método de conversión
    print("\n¿Qué método deseas usar para convertir el PDF?")
    print("1. PyMuPDF (recomendado, más rápido, no requiere Poppler)")
    print("2. pdf2image + Poppler (alternativo)")
    
    usar_pymupdf = True  # Cambiar a False para usar pdf2image+Poppler
    metodo = "PyMuPDF" if usar_pymupdf else "pdf2image+Poppler"
    print(f"\n✓ Usando método: {metodo}")
    
    # Inicializar extractor
    extractor = PDFTableExtractor(pdf_path, idioma='spa', usar_pymupdf=usar_pymupdf)
    
    try:
        # Procesar PDF (zoom más alto = mejor calidad pero más lento)
        tablas = extractor.procesar_pdf_completo(guardar_imagenes=True, zoom=3.0)
        
        # Mostrar y guardar resultados
        print(f"\n{'='*60}")
        print(f"RESULTADOS FINALES")
        print('='*60)
        print(f"\n✓ Total de tablas extraídas: {len(tablas)}\n")
        
        for tabla_info in tablas:
            print(f"\n{'─'*60}")
            print(f"Página {tabla_info['pagina']}, Tabla {tabla_info['tabla']}")
            print('─'*60)
            print(tabla_info['dataframe'])
            print(f"\nDimensiones: {tabla_info['dataframe'].shape[0]} filas x {tabla_info['dataframe'].shape[1]} columnas")
            
            # Guardar a CSV
            nombre_archivo = f"tabla_p{tabla_info['pagina']}_t{tabla_info['tabla']}.csv"
            tabla_info['dataframe'].to_csv(nombre_archivo, index=False, encoding='utf-8-sig')
            print(f"✓ Guardado en: {nombre_archivo}")
        
        print(f"\n{'='*60}")
        print("PROCESO COMPLETADO")
        print('='*60)
        
    except Exception as e:
        print(f"\n✗ ERROR durante el procesamiento: {e}")
        import traceback
        traceback.print_exc()
    
    finally:
        extractor.cerrar()

RuntimeError: Directory 'static/' does not exist

In [14]:
pip uninstall fitz

^C
Note: you may need to restart the kernel to use updated packages.
